#Задание 9(b)

##Критерий $\chi^2$:

In [ ]:
from scipy.integrate import quad
from scipy import optimize
import numpy as np
import scipy.stats


def integrate_normal(lower, upper, teta1, teta2):
  return quad(lambda x: 1/(np.sqrt(2 * np.pi) * teta2) * np.e ** (-(x - teta1) ** 2 / (2 * teta2 ** 2)), lower, upper)[0]

def func_to_optimize(tetas):
  res = 1
  selection = np.array([5, 8, 6, 12, 14, 18, 11, 6, 13, 7])
  for i in range(10):
    if(i == 0):
      res *= integrate_normal(- np.inf, 1, tetas[0], tetas[1]) ** selection[i]
    elif(i == 9):
      res *= integrate_normal(9, np.inf, tetas[0], tetas[1]) ** selection[i]
    else:
      res *= integrate_normal(i, i + 1, tetas[0], tetas[1]) ** selection[i]
  return -res



x0 = np.array([11, 13])
maximized = optimize.minimize(func_to_optimize, x0, method = 'nelder-mead')
selection = np.array([5, 8, 6, 12, 14, 18, 11, 6, 13, 7])
delta_wave = 0
tetas = maximized.x
print(tetas, "- Теты, при которых достигается максимум")
for i in range(10):
  if(i == 0):
    integral = integrate_normal(- np.inf, 1, tetas[0], tetas[1]) * 100
    delta_wave += (selection[i] - integral) ** 2 / integral
    print(integral)
  elif(i == 9):
    integral = integrate_normal(9, np.inf, tetas[0], tetas[1]) * 100
    delta_wave += (selection[i] - integral) ** 2 / integral
    print(integral)
  else:
    integral = integrate_normal(i, i + 1, tetas[0], tetas[1]) * 100
    delta_wave += (selection[i] - integral) ** 2 / integral
    print(integral)
print("Дельта с волной:", delta_wave)

print(scipy.stats.chi2(7).sf(9.8), "= p-value")

[5.28962548 2.67951583] - Теты, при которых достигается максимум
5.470000660686341
5.508092515463962
8.663522214644964
11.873887426058692
14.180762281175316
14.757614759419813
13.382679980262724
10.574976112618318
7.281532461692063
8.306931587977822
Дельта с волной: 9.802654592391548
0.2001934364129521 = p-value


##Критерий Колмогорова:

In [ ]:
selection = np.array([5, 8, 6, 12, 14, 18, 11, 6, 13, 7])

def find_parameter_mean(x):
  return np.mean(x)

def find_parameter_variance(slection_1):
  return np.sqrt(np.var(slection_1) * 100 / 99)

def count_delta(selection, teta1, teta2):
  mx = 0
  incr_selection = [0] * 10
  incr_selection[0] = 0
  for i in range(1, len(selection)):
    incr_selection[i] = selection[i - 1] + incr_selection[i - 1]
  for i in range(len(incr_selection) - 1):
    integral = scipy.stats.norm(teta1, teta2).cdf(i)
    if i != 9:
      if mx < max(abs(integral - incr_selection[i] / 100), abs(integral - incr_selection[i + 1] / 100)):
        mx = max(abs (integral - incr_selection[i] / 100), abs(integral - incr_selection[i + 1] / 100))
    else:
      if mx < abs(integral - incr_selection[i] / 100):
        mx = abs(integral - incr_selection[i] / 100)
    return 10 * mx

def bootstrap_par(teta1_waved, teta2_waved, n_bootstrap_samples=50000):
  bootstrap_deltas = []
  for i in range(n_bootstrap_samples):
    samples = np.random.normal(teta1_waved, teta2_waved, 100).astype(int)
    selection_intervals = [0] * 10
    for k in samples:
      if k < 0:
        k = 0
      elif k > 9:
        k = 9
      selection_intervals[k] += 1
    print(selection_intervals)
    bootstrap_deltas.append(count_delta(selection_intervals, find_parameter_mean(samples), find_parameter_variance(samples)))
    print(i, end = " ")
  return bootstrap_deltas


sel = []
for i in range(len(selection)):
  for j in range(selection[i]):
    sel.append(i)
teta1 = find_parameter_mean(sel)
teta2 = find_parameter_variance(sel)
delta = count_delta(selection, teta1, teta2)
print(selection)
print(delta, "- дельта для исходной выборки")
deltas = bootstrap_par(teta1, teta2)
deltas.sort()
s = 0
for i in range(50000):
  if delta > deltas[i]:
    s += 1
  else:
    break
print()

print(s / 50000, end = " ")
if s / 50000 <= 0.05:
  print("Отбрасываем")
elif s / 50000 > 0.05:
  print("Не имеем оснований отбросить")



Выходные данные были обрезаны до нескольких последних строк (5000).
45001 [7, 4, 10, 12, 28, 10, 12, 10, 5, 2]
45002 [8, 9, 9, 10, 15, 14, 15, 11, 6, 3]
45003 [7, 4, 14, 14, 9, 17, 10, 12, 8, 5]
45004 [6, 5, 8, 15, 20, 15, 12, 11, 7, 1]
45005 [9, 6, 10, 10, 20, 13, 10, 12, 4, 6]
45006 [4, 9, 8, 13, 19, 15, 10, 8, 10, 4]
45007 [3, 5, 12, 17, 15, 15, 9, 9, 9, 6]
45008 [7, 7, 15, 12, 13, 16, 12, 9, 5, 4]
45009 [4, 8, 13, 17, 15, 15, 9, 7, 6, 6]
45010 [4, 5, 11, 18, 19, 20, 11, 6, 3, 3]
45011 [7, 10, 14, 17, 7, 16, 11, 9, 3, 6]
45012 [6, 4, 18, 7, 14, 15, 14, 10, 3, 9]
45013 [5, 4, 10, 16, 22, 16, 8, 10, 2, 7]
45014 [8, 8, 13, 17, 19, 12, 10, 4, 5, 4]
45015 [6, 11, 18, 12, 20, 11, 11, 7, 3, 1]
45016 [5, 5, 11, 15, 16, 12, 16, 9, 4, 7]
45017 [8, 7, 12, 12, 14, 14, 15, 9, 4, 5]
45018 [11, 4, 6, 16, 12, 11, 17, 6, 8, 9]
45019 [7, 7, 9, 14, 11, 16, 18, 9, 6, 3]
45020 [10, 6, 8, 20, 7, 15, 14, 7, 7, 6]
45021 [6, 7, 6, 11, 18, 19, 11, 11, 8, 3]
45022 [5, 7, 8, 11, 22, 16, 16, 4, 4, 7]
45023 [7, 